In [ ]:
### 코랩 기준으로
# !pip install timezonefinder
# !pip install datatable

In [ ]:
### 데이터 빠르게 읽는 코드
import datatable as dt
def read_data_faster(file_path="/content/drive/MyDrive/DM_project/filtered_data.csv"):
  data = dt.fread(file_path)
  data = data.to_pandas()
  data = data.drop(new_data.columns[0], axis=1)
  return data
data = read_data_faster()

기본 전처리

In [ ]:
def preprocess_data(data):
  ### 미국, 캐나다 말고 다 빼기
  # FN, UM, MH 빼기
  data = data[(data["STATE"]!="FN") & (data["STATE"]!="UM") & (data["STATE"]!="MH")]

  ### AIRPORT_ID 기준으로 unknown 및 공항 아닌거 빼버림
  # H2O: 물에 떨어짐
  # RIGG: oil 어쩌구
  # PVT: 개인소유
  data = data[(data["AIRPORT_ID"]!="ZZZZ") & (data["AIRPORT_ID"]!="H2O") & (data["AIRPORT_ID"]!="RIGG") & (data["AIRPORT_ID"]!="PVT")]

  ### LATITUDE, LONGTITUDE 처리
  data["AIRPORT_LATITUDE"] = [x if len(x.split(","))==1 else x.split(",")[0] for x in data["AIRPORT_LATITUDE"]] # latitude 입력 형식 잘못된 부분 정정
  data["AIRPORT_LONGITUDE"] = data["AIRPORT_LONGITUDE"].astype(float)
  data["AIRPORT_LATITUDE"] = data["AIRPORT_LATITUDE"].astype(float)

  ### AC_CLASS, AC_MASS가 NAN인건 빼기
  # 세부 기종을 알 수 없으므로 MASS도 알 수 없다.
  drop_idx = data[(data["AC_CLASS"].isnull()) | (data["AC_MASS"].isnull())].index
  data.drop(drop_idx, inplace=True)

  ### DAMAGE_sum 생성
  damage_cols = ["DAM_RAD", "DAM_WINDSHLD", "DAM_NOSE", "DAM_ENG1", "DAM_ENG2", "DAM_ENG3", "DAM_ENG4",
                "DAM_PROP", "DAM_WING_ROT", "DAM_FUSE", "DAM_LG", "DAM_TAIL", "DAM_LGHTS", "DAM_OTHER"]
  def damage_sums(data, damage_cols=damage_cols):
    damage_sum = data[damage_cols[0]].astype(int)
    for i in range(1,len(damage_cols)):
      damage_sum = damage_sum + data[damage_cols[i]].astype(int)
    return damage_sum
  damage_sum = damage_sums(data)
  data["DAMAGE_sum"] = damage_sum

  ### DAMAGE_sum이 0이면 전부 DAMAGE_LEVEL이 N이다.
  ### DAMAGE_LEVEL 결측치는 전부 N으로 채웠다.
  ### 그중에 DAMAGE_sum이 1인 컬럼은 fatality, injury가 없고 엔진이 아닌 날개부분에 데미지가 있으므로 N으로 처리한다.
  data["DAMAGE_LEVEL"] = data["DAMAGE_LEVEL"].fillna("N")
  return data
data = preprocess_data(data)

TIME_OF_DAY 채우기
- 꼭!!!!! latitude, longtitude float화 해야함
- latitude는 포맷이 이상한게 3개 있어서 정제해줌

In [ ]:
### TIME이 있으면서 TIME_OF_DAY가 null인 데이터 매뉴얼하게 채우기
# 미국 해양대기청에서 발표한 공식으로 계산
# 총 5758개 채울 수 있다 (time이 null아니고 time_of_day가 null인것)
# 돌리기 전에 latitude, longtitude 꼭 float로 변환

## manual sunrise & sunset time calculation
from timezonefinder import TimezoneFinder
import pytz
from ast import Try
from math import pi, sin, cos, tan, acos, asin, atan, atan2
from datetime import datetime
import time
HH=0
MM=1

SUN_DIAMETER= 0.53
AIR_REF = (34.0/60.0)

def is_leap_year(year):
  if year%4==0 and year%100==0 and year%400!=0:
    return False
  elif year%4==0 and year%100!=0:
    return True
  elif year%4==0 and year%100==0 and year%400==0:
    return True
  return False

def get_julian_day(year, month, day):
  tmp=-7*(float(year) + (float(month)+9)/12) / \
    4.0+275.0*float(month) / 9.0 + float(day)
  tmp += float(year*367)
  return (tmp-730531.5 + 12.0 / 24.0)

def get_range_radian(x):
  b = float(x/(2*pi))
  a = float((2*pi) *(b-int(b)))
  if a<0:
    a = (2*pi) + a
  return a

def get_ha(lat, decl):
  dfo = pi/180.0 * (0.5*SUN_DIAMETER+ AIR_REF)
  if lat<0.0:
    dfo=-dfo
  fo = tan(decl+dfo) * tan(lat*pi/180.0)
  if fo>1.0:
    fo=1.0
  fo = asin(fo) + pi/2.0
  return fo

def get_sun_longitude(days):
  longitude = get_range_radian(
      280.461 * pi/180.0 + 0.9856474 * pi/180.0 * days
  )
  g = get_range_radian(357.528 * pi/180.0 + 0.9856003 * pi /180.0 * days)
  return get_range_radian(longitude + 1.915*pi/180.0 * sin(g)+0.02*pi/180.0*sin(2*g)), longitude

def convert_dtime_to_rtime(dhour):
  hour = int(dhour)
  minute = int((dhour-hour)*60)
  return hour, minute

def calculate_sunset_sunrise(year, month, day, latitude, longitude, timezone):
  days = get_julian_day(year, month, day)
  gamma, mean_longitude = get_sun_longitude(days)
  obliq = 23.439 * pi / 180.0 - 0.0000004 * pi/180.0 * days
  alpha = atan2(cos(obliq)*sin(gamma), cos(gamma))
  delta = asin(sin(obliq)*sin(gamma))
  ll = mean_longitude - alpha
  if mean_longitude<pi:
    ll += (2*pi)
  eq = 1440.0 * (1.0-ll/(2*pi))
  ha = get_ha(latitude, delta)
  sunrise = 12.0-12.0 * ha/pi + timezone-longitude/15.0 + eq/60.0
  sunset = 12.0+12.0 * ha/pi + timezone-longitude/15.0 + eq/60.0

  if sunrise > 24.0:
    sunrise -=24.0
  if sunset>24.0:
    sunset -= 24.0
  sunrise_time = [0,0]
  sunset_time = [0,0]
  sunrise_time[HH], sunrise_time[MM] = convert_dtime_to_rtime(sunrise)
  sunset_time[HH], sunset_time[MM] = convert_dtime_to_rtime(sunset)

  ret_sunrise = sunrise_time[HH] * 60 + sunrise_time[MM]
  ret_sunset = sunset_time[HH] * 60 + sunset_time[MM]
  return ret_sunrise, ret_sunset


## TIME_OF_DAY 결측값 채우는 함수
## Dawn, Day, Dusk, Night 기준은 if문 부분 참조("civil twilight" https://www.timeanddate.com/astronomy/different-types-twilight.html)
def create_time_of_day(df):
  idx_list = df.index.tolist()
  tod = []
  for i in idx_list:
    row = df.loc[i]
    year = row["INCIDENT_YEAR"]
    month = row["INCIDENT_MONTH"]
    date = row["INCIDENT_DATE"]
    tz = row["TIMEZONE"]
    latitude = row["AIRPORT_LATITUDE"]
    longitude = row["AIRPORT_LONGITUDE"]
    ret_sunrise, ret_sunset = calculate_sunset_sunrise(year, month, date, latitude, longitude, tz)
    dawn_start = ret_sunrise - 30
    dusk_end = ret_sunset + 30
    hour, min = map(int, row["TIME"].split(":"))
    minute_of_day = hour * 60 + min

    if dawn_start <= minute_of_day < ret_sunrise:
        tod.append("Dawn")
    elif ret_sunrise <= minute_of_day < ret_sunset:
        tod.append("Day")
    elif ret_sunset <= minute_of_day < dusk_end:
        tod.append("Dusk")
    else:
        tod.append("Night")
  return tod

def get_tod_filled_data(new_data):
  temp_data = new_data[["INCIDENT_YEAR",'INCIDENT_MONTH',"INCIDENT_DATE","TIME","AIRPORT_LATITUDE","AIRPORT_LONGITUDE","TIME_OF_DAY"]]
  imputed_idx = temp_data[(temp_data["TIME_OF_DAY"].isnull()) & (temp_data["TIME"].notnull())].index.tolist()
  drop_idx = temp_data[(temp_data["TIME"].isnull())].index
  temp_data.drop(drop_idx, inplace=True)
  temp_data["INCIDENT_DATE"] = temp_data["INCIDENT_DATE"].dt.day

  tf = TimezoneFinder()
  def find_timezone(longitude, latitude):
    timezone_str = tf.timezone_at(lng=longitude, lat=latitude)
    timezone = pytz.timezone(timezone_str)
    utc_offset = timezone.utcoffset(datetime.now()).total_seconds() / 3600
    return utc_offset
  temp_data["TIMEZONE"] = [find_timezone(x[0], x[1]) for x in zip(temp_data["AIRPORT_LONGITUDE"], temp_data["AIRPORT_LATITUDE"])]

  temp2 = temp_data[temp_data["TIME_OF_DAY"].isnull()]
  time_of_day = create_time_of_day(temp2)
  for i in range(len(imputed_idx)):
    new_data["TIME_OF_DAY"][imputed_idx[i]] = time_of_day[i]

  # 꼭 return 해야하나?
  return new_data

## 실행 코드
data = get_tod_filled_data(data)

In [ ]:
### 변수 분류
# 안쓰는거 (첫째줄부터 항공편 상세정보(AIRCRAFT는 결측치 처리에 쓸 수 있음), 엔진 관련 정보("TYPE_ENG"은 쓴다), 좌표랑 중복정보인거, 사후에만 알 수 있는 정보, 보고와 관련된 컬럼)
# TRANSFER는 뭔지 모르겠다
no_usage = ["OPID","AIRCRAFT","REG","FLT","RUNWAY","AMA","AMO",
            "EMO", "EMA",
            "AIRPORT","AIRPORT_ID",
            "REMAINS_COLLECTED","REMAINS_SENT", "PHASE_OF_FLIGHT",
            "BIRD_BAND_NUMBER","COMMENTS","REPORTED_NAME","REPORTED_TITLE","SOURCE","PERSON","LUPDATE","TRANSFER"]

# EDA에 쓰는거
# LOCATION, REMARKS는 이상치에 대한 상세 정보를 알기 위해 남겨놓는다 (텍스트 타입)
for_eda = ["LOCATION","HEIGHT","SPEED","DISTANCE","SPECIES_ID", "SPECIES", "BIRDS_SEEN", "BIRDS_STRUCK", "SIZE", "WARNED", "REMARKS"]

# 반응변수로 만들어야 하는거
# 순서대로 aircraft damage, effect on flight, cost, 사상자
# 그 밖에 STR, DAM, ING로 시작하는거
# EFFECT, EFFECT_OTHER는 텍스트 (대부분 NaN)
for_response = ["DAMAGE","OTHER_SPECIFY",
                "EFFECT", "EFFECT_OTHER", "AOS",
                "COST_REPAIRS","COST_OTHER","COST_REPAIRS_INFL_ADJ","COST_OTHER_INFL_ADJ",
                "NR_INJURIES","NR_FATALITIES"]